In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
import nltk
stop_words_l=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 4.0 MB 62.6 MB/s 
     |████████████████████████████████| 1.2 MB 50.5 MB/s 
     |████████████████████████████████| 77 kB 3.7 MB/s 
     |████████████████████████████████| 880 kB 57.3 MB/s 
     |████████████████████████████████| 596 kB 64.1 MB/s 
     |████████████████████████████████| 6.6 MB 58.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0ce23dd3cf075ba1fb491525d6424dba7f090a2f00e0588487cb41c32ae48e1b
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d919e5ba3acc70495be48cb489e2c423a5c5b86f64ab78329cbc9f33739fd876
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [ ]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import timeit

In [ ]:
models = ['stsb-roberta-large',
'stsb-roberta-base',
'stsb-bert-large',
'stsb-distilbert-base',
'stsb-bert-base',
'paraphrase-xlm-r-multilingual-v1',
'paraphrase-distilroberta-base-v1',
'nli-bert-large',
'nli-distilbert-base',
'nli-roberta-large',
'nli-bert-large-max-pooling',
'nli-bert-large-cls-pooling',
'nli-distilbert-base-max-pooling',
'nli-roberta-base',
'nli-bert-base-max-pooling',
'nli-bert-base',
'nli-bert-base-cls-pooling'
]
scores = []
compute_times = []
labels = []
order = []

In [ ]:
!pip install stop-words
from stop_words import get_stop_words

In [ ]:
for i in models:


  model = SentenceTransformer(i)
  path='/content/drive/MyDrive/task-c/task-C-cs/train'
  start = timeit.default_timer()
  l1=[]
  compute_order = []
  files=os.listdir(path)
  for x in tqdm(files, position=0, leave=True):
    path1=path+'/'+x+'/'
    f=os.listdir(path1)
    if len(f)==1:
        continue
    text1=open(path1+f[0],encoding="utf8").read()
    text2=open(path1+f[1],encoding="utf8").read()
    text=[text1,text2]
    documents_df=pd.DataFrame(text,columns=['documents'])
    stop_words_l=get_stop_words('czech')
    documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]','',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]','',w).lower() not in stop_words_l) )

    # Two lists of sentences
    sentences1 = documents_df.documents_cleaned[0]
    sentences2 = documents_df.documents_cleaned[1]

    #Compute embedding for both lists
    embeddings1 = model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarits
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    cosine_scores = cosine_scores.cpu()
    compute_order.append(int(re.sub("[^0-9]", "", x)))
    l1.append(np.array(cosine_scores[0][0]))
  scores.append(l1)
  order.append(compute_order)
  stop = timeit.default_timer()
  compute_times.append(stop-start)

compute_times

100%|██████████| 780/780 [00:29<00:00, 26.37it/s]


[319.91277289500067,
 29.159183904999736,
 57.351678306000395,
 19.411789750000025,
 29.517221594000148,
 29.090188870999555,
 19.5865731089998,
 57.35627500699957,
 19.06662631400104,
 57.480448658998284,
 57.43127163200006,
 57.50310601599995,
 19.77431591399909,
 29.715462729000137,
 29.889233382000384,
 29.35593353400145,
 29.59047722499963]

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/task-c/CS_train_task_c.tsv', sep = '\t')
df.Directory = df.Directory.str.replace("[^0-9]", "").astype(int)
df = df.sort_values(by='Directory')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Directory,File 1,File 2,Label,Task
0,1,2,meeting_cs_train_008_minutes_ORIG.deidentified...,meeting_cs_train_015_minutes_GENER_annot18.dei...,False,C
1,2,4,meeting_cs_train_003_minutes_GENER_annot18.dei...,meeting_cs_train_013_minutes_GENER_annot05.dei...,False,C
2,3,6,meeting_cs_train_033_minutes_GENER_annot02.dei...,meeting_cs_train_021_minutes_ORIG.deidentified...,False,C
3,4,9,meeting_cs_train_020_minutes_ORIG.deidentified...,meeting_cs_train_015_minutes_GENER_annot05.dei...,False,C
4,5,10,meeting_cs_train_025_minutes_ORIG.deidentified...,meeting_cs_train_001_minutes_GENER_annot18.dei...,False,C
...,...,...,...,...,...,...
775,776,1642,meeting_cs_train_013_minutes_ORIG.deidentified...,meeting_cs_train_002_minutes_GENER_annot19.dei...,False,C
776,777,1645,meeting_cs_train_019_minutes_GENER_annot18.dei...,meeting_cs_train_027_minutes_GENER_annot19.dei...,False,C
777,778,1646,meeting_cs_train_031_minutes_GENER_annot19.dei...,meeting_cs_train_015_minutes_GENER_annot05.dei...,False,C
778,779,1647,meeting_cs_train_007_minutes_GENER_annot05.dei...,meeting_cs_train_025_minutes_ORIG.deidentified...,False,C


In [ ]:
from sklearn.metrics import roc_auc_score
roc = []
i = 0
for x in order:
  df2 = pd.DataFrame({'File' : x, 'Score': scores[i]})
  df2 = df2.sort_values(by='File')
  temp = df[df.Directory.isin(df2.File)]
  df2.loc[df2.Score>=0.75, 'Label']=True
  df2.loc[df2.Score<0.75, 'Label']=False
  roc.append(accuracy_score(temp.Label, df2.Label.astype(bool)))
  i = i+1

In [ ]:
roc

[0.823076923076923,
 0.3564102564102564,
 0.8166666666666667,
 0.4076923076923077,
 0.33205128205128204,
 0.6423076923076924,
 0.8666666666666667,
 0.19615384615384615,
 0.28717948717948716,
 0.2717948717948718,
 0.09230769230769231,
 0.30641025641025643,
 0.19615384615384615,
 0.22179487179487178,
 0.05128205128205128,
 0.2602564102564103,
 0.2653846153846154]

In [ ]:
compute_times

[319.91277289500067,
 29.159183904999736,
 57.351678306000395,
 19.411789750000025,
 29.517221594000148,
 29.090188870999555,
 19.5865731089998,
 57.35627500699957,
 19.06662631400104,
 57.480448658998284,
 57.43127163200006,
 57.50310601599995,
 19.77431591399909,
 29.715462729000137,
 29.889233382000384,
 29.35593353400145,
 29.59047722499963]